In [11]:
import os
import sys
import time
import datetime
import netCDF4
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from sklearn.cluster import DBSCAN

In [13]:
s3bands = [
    'B1-400',
    'B2-412.5',
    'B3-442.5',
    'B4-490',
    'B5-510',
    'B6-560',
    'B7-620',
    'B8-665',
    'B9-673.75',
    'B10-681.25',
    'B11-708.75',
    'B12-753.75',
    'B16-778.75',
    'B17-865',
    'B18-885',
    'B21-1020'
]

def datafolder_2_list(path):
    return [f for f in path.iterdir() if f.suffix == '.csv']


def open_radiometry(path):
    old_names = [f'Oa{str(i).zfill(2)}_reflectance:float' for i in range(1, 22)]
    [old_names.remove(f'Oa{str(i).zfill(2)}_reflectance:float') for i in range(13,16)]
    [old_names.remove(f'Oa{str(i).zfill(2)}_reflectance:float') for i in range(19,21)]
    df = pd.read_csv(path).rename(columns={old_names[i]: s3bands[i] for i in range(16)})    
    return df


def normalize(df, bands, norm_band):
    df = df.copy()
    df[bands] = df[bands].to_numpy() - df[norm_band].to_numpy()[..., None]
    return df


def clip_negatives(df, bands, threshold=-0.1):
    return df.loc[~((df[bands] < -0.1).any(axis=1))]


def clean_radiometry(df, min_threshold=-0.1, norm_band='B21-1020'):

    if norm_band is not None:
        df = normalize(df, s3bands, 'B21-1020')

    df = clip_negatives(df, s3bands, -0.1)

    return df


def load_radiometries(path, min_pixels=10):

    csvs = [f for f in path.iterdir()]

    radiometries = {}

    for i, csv in enumerate(csvs):
        df = open_radiometry(csvs[i])
        if len(df) > min_pixels:
            radiometries[csv.stem[16:24]] = df
            
    return radiometries


def calc_nd_index(df, band1, band2, column_name='nd_index'):
    idx = (df[band1]-df[band2])/(df[band1]+df[band2])
    df[column_name] = idx
    
    
def db_scan(df, bands, column_name='cluster', eps=0.1, min_samples=5):
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(df[bands])
    df[column_name] = clustering.labels_
    

def power(x, a, b, c):return a*(x)**(b) + c


def SPM_GET_Amazon(b665, b865, cutoff_value=0.027, cutoff_delta=0.007, low_params=None, high_params=None, debug=False):

    if debug:
        pdb.set_trace()

    if cutoff_delta == 0:
        transition_coef = np.where(b665<=cutoff_value, 0, 1)

    else:
        transition_range = (cutoff_value - cutoff_delta, cutoff_value + cutoff_delta)
        transition_coef = (b665-transition_range[0])/(transition_range[1]-transition_range[0])

        transition_coef = np.clip(transition_coef, 0, 1)


    # if params are not passed, use default params obtained from the Amazon dataset
    low_params = [2.79101975e+05, 2.34858344e+00, 4.20023206e+00] if low_params is None else low_params
    high_params = [848.97770516,   1.79293191,   8.2788616 ] if high_params is None else high_params

    #low = Fit.power(b665, *low_params).fillna(0)
    #high = Fit.power(b865/b665, *high_params).fillna(0)

    low = power(b665, *low_params).fillna(0)
    high = power(b865/b665, *high_params).fillna(0)


    spm = (1-transition_coef)*low + transition_coef*high
    return spm


# https://stackoverflow.com/questions/29387137/how-to-convert-a-given-ordinal-number-from-excel-to-a-date
def from_excel_ordinal(ordinal, _epoch0=datetime.datetime(1899, 12, 31)):
    if ordinal >= 60:
        ordinal -= 1  # Excel leap year bug, 1900 is not a leap year!
    return (_epoch0 + datetime.timedelta(days=ordinal)).replace(microsecond=0)


pd.options.mode.chained_assignment = None  # default='warn'

def create_time_series_cluster(radiometries, bands=['B17-865', 'B21-1020'], eps=0.01, min_samples=5, save_folder=None):
    df = pd.DataFrame()

    for date, radiometry in radiometries.items():

        # clean some outliars  using the indices
        calc_nd_index(radiometry, 'B6-560', 'B21-1020', column_name='mndwi')  # Green / SWIR
        calc_nd_index(radiometry, 'B6-560', 'B17-865', column_name='ndwi')  # Green / IR

        valid_mndwi = (radiometry['mndwi'] > -0.99) & (radiometry['mndwi'] < 0.99)
        valid_ndwi = (radiometry['ndwi'] > -0.99) & (radiometry['ndwi'] < 0.99)

        radiometry = radiometry[valid_mndwi & valid_ndwi]

        if len(radiometry) < min_samples:
            continue 
        
        db_scan(radiometry, bands, eps=eps, min_samples=min_samples)
        clusters = radiometry.groupby(by='cluster').mean()
        
        if save_folder:
            plt.scatter()
            plt.savefig(save_folder, bbox_inches='tight')
        
        # drop the noise
        clusters.drop(-1, inplace=True, errors='ignore')

        # if there is at least 1 valid cluster
        if len(clusters) > 0:
            df = df.append(clusters[clusters['B21-1020'] == clusters['B21-1020'].min()].iloc[0].rename(date))
        
    
    df['Datestr'] = [f'{i[:4]}-{i[4:6]}-{i[6:]}' for i, row in df.iterrows()]
    df['Date'] = pd.to_datetime(df['Datestr'])
    df.sort_values(by='Date', inplace=True)
    df.reset_index(inplace=True)
    
    return df


def just_open_radiometry(path):
    df = pd.read_csv(path)    
    return df


def just_load_radiometries(path, min_pixels=3):

    csvs = [f for f in path.iterdir()]

    radiometries = {}

    for i, csv in enumerate(csvs):
        df = just_open_radiometry(csvs[i])
        if len(df) > min_pixels:
            radiometries[csv.stem[16:24]] = df
            
    return radiometries


def plot_scattercluster(event_df, col_x='B17-865', col_y='B8-665', col_color='T865:float', cluster_col='cluster', nx=None, ny=None, mplcolormap='viridis', title=None, savepath=None, dpi=100):
    
    plt.rcParams['figure.figsize'] = [14, 5.2]
    fig, (ax1, ax2) = plt.subplots(1, 2)
    
    if title:
        fig.suptitle(title)

    skt1 = ax1.scatter(event_df[col_x], event_df[col_y], c=event_df[col_color], cmap=mplcolormap)
    cbar = fig.colorbar(skt1, ax=ax1)
    cbar.set_label(col_color)

    # Get unique names of clusters
    uniq = list(set(event_df[cluster_col]))

    # iterate to plot each cluster
    for i in range(len(uniq)):
        indx = event_df[cluster_col] == uniq[i]
        ax2.scatter(event_df[col_x][indx], event_df[col_y][indx], label=uniq[i])
    
    # Add x,y annotation
    if nx:
        ax1.plt.plot(nx, ny,
                     marker='D',
                     markersize=20,
                     markerfacecolor="None",
                     markeredgecolor='k')
    
    ax1.set_xlabel(col_x)
    ax1.set_ylabel(col_y)
    ax2.set_xlabel(col_x)
    
    plt.legend()
    
    if savepath:
        plt.savefig(savepath, dpi=dpi, bbox_inches='tight')
        plt.close(fig)
    else:
        plt.show()
        
        
import matplotlib
import matplotlib.cm as cm
sys.path.append('../../')
from nc_explorer import NcExplorer

ncxp = NcExplorer()

def s3l2_custom_reflectance_plot(df, figure_title, save_title=None):
    
    colnms = ['Oa01_reflectance:float',
             'Oa02_reflectance:float',
             'Oa03_reflectance:float',
             'Oa04_reflectance:float',
             'Oa05_reflectance:float',
             'Oa06_reflectance:float',
             'Oa07_reflectance:float',
             'Oa08_reflectance:float',
             'Oa09_reflectance:float',
             'Oa10_reflectance:float',
             'Oa11_reflectance:float',
             'Oa12_reflectance:float',
             'Oa16_reflectance:float',
             'Oa17_reflectance:float',
             'Oa18_reflectance:float',
             'Oa21_reflectance:float']
    
    # creating a list of lists containing the reflectance of every band in each df row.
    spectral_lib = []
    for n,i in enumerate(df):
        plotngdata = [df[c].iloc[n] for c in colnms]
        spectral_lib.append(plotngdata)
    
    # creating color scale based on T865
    lst = df['T865:float']
    minima = min(lst)
    maxima = max(lst)
    norm = matplotlib.colors.Normalize(vmin=minima, vmax=maxima, clip=True)
    mapper = cm.ScalarMappable(norm=norm, cmap=cm.viridis)
    
    # create a list with the value in (nm) of the 16 Sentinel-3 bands for L2 products.
    s3_bands_tick = list(ncxp.s3_bands_l2.values())

    # create a list with the name of the 16 Sentinel-3 bands for L2 products.
    s3_bands_tick_label = list(ncxp.s3_bands_l2.keys())    
    
    plt.rcParams['figure.figsize'] = [12, 6] 
    

    fig = plt.figure()
    fig.show()
    ax1 = fig.add_subplot(111)
      
    ax1.set_xlabel('Wavelenght (nm)')
    ax1.set_ylabel('Reflectance')
    ax1.set_title(figure_title, y=1, fontsize=16)

    for n, spec_curve in enumerate(spectral_lib):
        t865c = mapper.to_rgba(df['T865:float'][n])
        ax1.plot(s3_bands_tick, spec_curve, alpha=0.4, c=t865c)

    ax1.axhline(y=0, xmin=0, xmax=1, linewidth=0.5, color='black', linestyle='--')
    ax1.set_xticks(s3_bands_tick)
    ax1.set_xticklabels(s3_bands_tick)
    ax1.tick_params(labelrotation=90, labelsize='small')
#     ax1.legend()

    ax2 = ax1.twiny()
    ax2.plot(s3_bands_tick, [0]*(len(s3_bands_tick)), alpha=0.0)
    ax2.set_xticks(s3_bands_tick)
    ax2.set_xticklabels(s3_bands_tick_label)
    ax2.tick_params(labelrotation=90, labelsize='xx-small')
    ax2.set_title('Sentinel-3 Oa Bands', y=0.93, x=0.12, fontsize='xx-small')
    
    if save_title:
        plt.savefig(save_title, dpi=300)
    else:
        plt.show()
        
        
# concentração = 759,12 * (NIR /RED)^1,92
def SPM_MOD(NIR,RED):
    return 759.12*((NIR/RED)**1.92)


def SPM_GETAMZmod(band665, band865, cutoff_value=0.027, cutoff_delta=0.007, low_params=None, high_params=None, debug=False):
    
    b665 = band665/np.pi
    b865 = band865/np.pi
    
    if debug:
        pdb.set_trace()

    if cutoff_delta == 0:
        transition_coef = np.where(b665<=cutoff_value, 0, 1)

    else:
        transition_range = (cutoff_value - cutoff_delta, cutoff_value + cutoff_delta)
        transition_coef = (b665-transition_range[0])/(transition_range[1]-transition_range[0])

        transition_coef = np.clip(transition_coef, 0, 1)


    # if params are not passed, use default params obtained from the Amazon dataset
    low_params = [2.79101975e+05, 2.34858344e+00, 4.20023206e+00] if low_params is None else low_params
    high_params = [848.97770516,   1.79293191,   8.2788616 ] if high_params is None else high_params

    #low = Fit.power(b665, *low_params).fillna(0)
    #high = Fit.power(b865/b665, *high_params).fillna(0)

    low = power(b665, *low_params).fillna(0)
    #high = power(b865/b665, *high_params).fillna(0)
    high = SPM_MOD(b865,b665)


    spm = (1-transition_coef)*low + transition_coef*high
    return spm


def load_sediments(path2xlsx_file, sheet_name='sedimentos'):
    df_sed = pd.read_excel(path2xlsx_file, sheet_name=sheet_name)
    # Fix excel dates from ordinal (40603) to datetime.datetime(2011, 3, 1, 0, 0)
    df_sed['pydate'] = [from_excel_ordinal(d) for d in df_sed['Data']]
    df_sed.sort_values(by='pydate', inplace=True)
    return df_sed

Declaring class instance from: SEN3R:nc_explorer
Input NetCDF file folder not set. Proceed at your own risk.
initialize set to False, ignoring image geometries. This can be later done manually by calling initialize_geometries() after properly setting the nc_folder.


## SPM

In [14]:
manacapuru_sed_xls = Path('D:/processing/linux/14100000_manacapuru_v15_sedimentos.xlsx')
df_manacapuru_sed = load_sediments(manacapuru_sed_xls)

In [16]:
borba_sed_xls = Path('D:/processing/linux/15860000_borba_madeira_v15_sedimentos.xlsx')
df_borba_sed = load_sediments(borba_sed_xls)

In [17]:
obidos_sed_xls = Path('D:/processing/linux/17050001_obidos_v15b_sedimentos.xlsx')
df_obidos_sed = load_sediments(obidos_sed_xls)
df_obidos_sed.drop(df_obidos_sed.tail(1).index,inplace=True)